---
date: '2025-05-07T17:48:16+08:00'
title: 'Remove Bscmake'
---

In [1]:
from surun_tools.glob1 import glob_files

In [2]:
glob_files("*.vcxproj")

[WindowsPath('C:/src/surun/src/PC/InstallSuRun.vcxproj'),
 WindowsPath('C:/src/surun/src/PC/SuRun.vcxproj'),
 WindowsPath('C:/src/surun/src/PC/SuRunExt.vcxproj'),
 WindowsPath('C:/src/surun/tests/TestScreenshot/TestScreenshot.vcxproj'),
 WindowsPath('C:/src/surun/tests/TestScreenshotSuRun/TestScreenshotSuRun.vcxproj')]

In [4]:
lines = []
for i in glob_files("*.vcxproj"):
    content = i.read_text(encoding="utf8")
    for line in content.split('\n'):
        if "bscmake" in line.lower():
            lines.append(line)
lines[:3]

['    <Bscmake>', '    </Bscmake>', '    <Bscmake>']

In [5]:
import xml.etree.ElementTree as ET

def remove_bscmake_nodes(xml_file):
    """
    Removes all nodes with the name 'bscmake' and their contents from an XML file.

    Args:
        xml_file (str): The path to the XML file.
    """
    try:
        tree = ET.parse(xml_file)
        root = tree.getroot()

        nodes_to_remove = root.findall('bscmake')
        for node in nodes_to_remove:
            root.remove(node)

        tree.write(xml_file, encoding='utf-8', xml_declaration=True)
        print(f"Successfully removed all 'bscmake' nodes from '{xml_file}'.")

    except FileNotFoundError:
        print(f"Error: File not found at '{xml_file}'.")
    except ET.ParseError:
        print(f"Error: Could not parse the XML file '{xml_file}'.")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")

In [ ]:
for file_path in glob_files("*.vcxproj"):
    remove_bscmake_nodes(file_path)